## 小練習
請使用社群偵測法，找出2015年6月28號當天有哪些文章主題，並列舉該主題下的文章，文章數量與前十大主題關鍵字 (https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/20150628news.xlsx)

In [3]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/20150628news.xlsx', index_col = 0)

### 斷字斷詞 (Jieba) 將中文斷詞以後, 轉變成英文斷詞方式

In [5]:
import jieba
jieba.load_userdict('userdict.txt')
corpus = []
titles = []
for idx, rec in news.iterrows():
    corpus.append(' '.join(jieba.cut(rec.get('description'))))
    titles.append(rec.get('title'))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 0.699 seconds.
Prefix dict has been built succesfully.


### 建立詞頻矩陣  

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [10]:
X

<147x11138 sparse matrix of type '<class 'numpy.longlong'>'
	with 22639 stored elements in Compressed Sparse Row format>

### 計算距離(cosine distance)

In [11]:
from sklearn.metrics.pairwise import cosine_distances
cs = cosine_distances(X)

In [12]:
cs.shape

(147, 147)

### 文章分群  (社群偵測法)

In [56]:
m = (cs < 0.8).astype(int)

In [57]:
import networkx as nx
G = nx.from_numpy_array(m)

In [58]:
G

In [59]:
import community
clusters = community.best_partition(G)

### 表列每一群的文章數量與代表關鍵字

In [60]:
from collections import Counter
c = Counter(list(clusters.values()))

In [61]:
c.most_common(10)

[(0, 36),
 (9, 17),
 (1, 11),
 (31, 7),
 (10, 4),
 (60, 3),
 (2, 2),
 (7, 2),
 (23, 2),
 (32, 2)]

In [62]:
import numpy as np
np_group = np.array(list(clusters.values()))

In [67]:
np_titles = np.array(titles)
np_titles[np_group == 31].tolist()

['旺報觀點－鄭弘儀！別讓台灣人背黑鍋！',
 '旺報觀點－紀念抗戰 台日中獲和解轉機',
 '紀念抗戰 學者：反省、避免戰爭',
 '多位台老兵 受邀赴陸閱兵',
 '首例 馬頒紀念章給新四軍老兵',
 '鄭弘儀反紀念抗戰 陸批數典忘祖',
 '中間選民為洪秀柱按讚']

In [77]:
np_features = np.squeeze(np.array(X[np_group==0].sum(axis = 0)))
len(np_features)

np.argsort(np_features)[::-1]

array([ 2292, 10168,  2595, ...,  7129,  7128,     0])

In [81]:
np_features_names = np.array(vectorizer.get_feature_names())
','.join(np_features_names[np.argsort(np_features)[::-1][0:10]].tolist())

'北市,醫院,台北市,樂園,馬偕,淡水,八仙,桃園市,臺北,粉塵'

In [82]:
for group, cnt in c.most_common(5):
    print('第{}群 - {}篇文章'.format(group, cnt))
    np_features = np.squeeze(np.array(X[np_group==group].sum(axis = 0)))
    print('代表關鍵字:',','.join(np_features_names[np.argsort(np_features)[::-1][0:10]].tolist()))
    print(np_titles[np_group == group].tolist()[0])
    print('====================================')

第0群 - 36篇文章
代表關鍵字: 北市,醫院,台北市,樂園,馬偕,淡水,八仙,桃園市,臺北,粉塵
八仙塵爆  五相關人依公共危險重傷害法辦
第9群 - 17篇文章
代表關鍵字: 最佳,專輯,公司,有限,音樂,演唱,國語,金曲獎,股份,蔡依林
蔡依林淚奪金曲 錦榮傳訊恭喜
第1群 - 11篇文章
代表關鍵字: 希臘,元區,經濟,政府,可能,表示,問題,民眾,債務,債權國
希臘國內三分一自動櫃員機現金短缺
第31群 - 7篇文章
代表關鍵字: 抗戰,國民黨,大陸,台灣,紀念,老兵,勝利,鄭弘儀,戰爭,八路軍
旺報觀點－鄭弘儀！別讓台灣人背黑鍋！
第10群 - 4篇文章
代表關鍵字: 中國,美國,中資,上市,企業,投資,公司,外資,台灣,a股
風評：陷入獵巫遊戲的中資認定


## 文章分類

In [83]:
import pandas
news = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/taipeifubon/master/data/20171214news.xlsx', index_col = 0)

In [116]:
import jieba
jieba.load_userdict('userdict.txt')
titles = []
tags   = []
corpus = []
for idx, rec in news[news['category'].isin(['社會', '娛樂','政治'])].iterrows():
    titles.append(rec.get('title'))
    tags.append(rec.get('category'))
    corpus.append(' '.join(jieba.cut(rec.get('content'))))

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [118]:
X

<331x14756 sparse matrix of type '<class 'numpy.longlong'>'
	with 45395 stored elements in Compressed Sparse Row format>

In [119]:
y = tags

In [120]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y,train_titles, test_titles = train_test_split(X,y,titles ,test_size = 0.2, random_state = 42)

In [121]:
train_X.shape

(264, 14756)

In [122]:
test_X.shape

(67, 14756)

In [123]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_X, train_y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [124]:
predicted = clf.predict(test_X)

In [125]:
import numpy
target = numpy.array(test_y)
predicted == target 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [126]:
(predicted == target).sum() / len(test_y)

0.9850746268656716

In [127]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(target, predicted)

0.9850746268656716

In [128]:
print(clf.classes_)
confusion_matrix(target, predicted)

['娛樂' '政治' '社會']


array([[24,  0,  0],
       [ 0, 18,  0],
       [ 1,  0, 24]])

In [129]:
np_test_titles = np.array(test_titles)
np_test_titles[predicted != target]

array(['小心！在美結婚台灣沒登記\u3000偷腥照樣能捉姦'], dtype='<U36')

In [108]:
article = '''
〔記者徐郁雯／台北報導〕張韶涵近年來把演藝重心轉往中國，她日前與前師妹郭靜同台，演唱過去與死對頭范瑋琪合唱歌曲《如果的事》，只見張韶涵一臉尷尬，硬唱100秒後馬上喊卡，讓網友看了啼笑皆非。
張韶涵難得與郭靜同台，本來要唱合唱過的歌曲《仨人》，但張韶涵坦言歌詞有點記不起來，郭靜反問要唱什麼？張韶涵回應：「我的歌！」郭靜表示：「《如果的事》嗎？」只見張韶涵表情尷尬，乾笑說：「喔…好。」
前奏一下，張韶涵、郭靜互看秒噴笑，敬業唱到一半時，張韶涵忽然受不了笑說：「好就這樣子，因為Delay了，掰掰。」隨後與郭靜擁抱、匆匆下台，但幽默氛圍仍受到不少討論。
'''

In [113]:
article_vec = vectorizer.transform([' '.join(jieba.cut(article))])

In [114]:
clf.predict(article_vec)

array(['娛樂'], dtype='<U2')

## 文章正負情緒判斷